In [1]:
import pytz
import time
import os
import pymongo
from pax import core

client = pymongo.MongoClient()
runs_collection = client['run']['runs']

pax_ini_config_path = '/home/xams/xams/PMT12_gain/daq_to_raw_PMT12_gain.ini'
data_directory = '/home/xams/xams/PMT12_gain/K_series/'
done_field_name = 'event_building_complete'
prefab_ini_folder = '/home/xams/xams/daqcontrol/ini/'

In [2]:
ininames = sorted([x for x in os.listdir(prefab_ini_folder) if x.endswith('.ini')])


In [3]:
ininames.insert(0, ininames.pop(ininames.index('default_config.ini')))


In [4]:
ininames

['default_config.ini', 'PMT12_gain.ini', 'baseline.ini']

In [8]:
#Hard coding to set processing to done
def set_as_processed(name, done_field_name = done_field_name, set_as =True):
    runs_collection.find_one_and_update({'name': name},
                                   {'$set': {done_field_name: set_as}})
set_as_processed('170203_090152')

In [11]:
set_as_processed('170206_095041', done_field_name='processing_status', set_as = 'done')

In [75]:
runs = list(runs_collection.find())
run_doc = runs[0]

In [76]:
with open(os.path.join('/home/xams/', 'run_db_backup_170202.pkl'), mode='wb') as outfile:
    #pickle.dump(run_doc, outfile)
    pickle.dump(runs, outfile)

In [77]:

from prettytable import PrettyTable
run_doc_human = run_doc.copy()
run_doc_human_ini = run_doc_human['ini']
for key, val in run_doc_human_ini.items():
    run_doc_human['ini_' + key] = val
for reg_dict in run_doc_human_ini['registers']:
    run_doc_human['ini_registers_' + reg_dict['register']] = reg_dict
del(run_doc_human['ini'])
del(run_doc_human['ini_registers'])

t = PrettyTable(['key', 'value'])
for key, val in sorted(run_doc_human.items()):
   t.add_row([key, val])
t.align = "l"
with open('/home/xams/blabla.txt', 'w') as f:
    f.write(str(t))

In [78]:
import numpy as np
np.sort(list(run_doc_human.keys()))

array(['_id', 'comment', 'end', 'event_building_complete',
       'ini_baseline_mode', 'ini_blt_size', 'ini_boards',
       'ini_compression', 'ini_links', 'ini_mongo',
       'ini_occurrence_integral', 'ini_processing_mode',
       'ini_processing_num_threads', 'ini_processing_readout_threshold',
       'ini_registers_1028', 'ini_registers_1080', 'ini_registers_1328',
       'ini_registers_1380', 'ini_registers_1428', 'ini_registers_1480',
       'ini_registers_8000', 'ini_registers_800C', 'ini_registers_8020',
       'ini_registers_809C', 'ini_registers_8100', 'ini_registers_810C',
       'ini_registers_8114', 'ini_registers_8120', 'ini_registers_EF00',
       'ini_registers_EF14', 'ini_registers_EF18', 'ini_registers_EF1C',
       'ini_registers_EF24', 'ini_run_start', 'ini_write_mode', 'name',
       'start'], 
      dtype='<U32')

In [79]:
t = PrettyTable(['key', 'value'])
for key, val in sortedrun_doc_human.items()):
   t.add_row([key, val])
with open('/home/xams/blabla.txt', 'w') as f:
    f.write(str(t))

SyntaxError: invalid syntax (<ipython-input-79-2173c90b7362>, line 2)

In [ ]:
import json
import pickle
with open(os.path.join('/home/xams/', 'run_doc.pkl'), mode='w') as outfile:
    #pickle.dump(run_doc, outfile)
    json.dump(run_doc, outfile)

In [ ]:
pickle.load(
    open('/home/xams/xams/pmtdata/170202_104333/run_doc.pkl', 'rb')), 

In [ ]:
try:
    register_list = runs[0]['ini']['registers']
    channels_enabled_dict = next(item for item in register_list if item["register"] == "8120")
    channels_enabled_str = channels_enabled_dict['value']
    binary_string = bin(int(channels_enabled_str, 16))[2:]
    channel_enabled_list = [ch_n for ch_n, enabled in enumerate(reversed(binary_string)) if enabled=='1']
except:
    print('WARNING did not correctly read channel enable mask, using [0,3]...')
    channel_enabled_list = [0,3]

In [ ]:
channels_enabled_str

In [ ]:
(runs[0]['end'] - runs[0]['start']).total_seconds() / 60

In [ ]:
import datetime
datetime.datetime.utcnow().strftime('%y%m%d_%H%M%S')

In [ ]:
while True:
    # Get all runs for which events have not been built yet
    runs_todo = list(runs_collection.find({done_field_name: {'$exists': False}}))
    print("%d runs to do" % len(runs_todo))
    
    if not runs_todo:
        print("No more runs to process, waiting a while...")
        time.sleep(10)
        continue
    
    for run_doc in runs_todo:
        _id = run_doc['_id']
        
        # Create a timestamp-based run label, which we use to name the output folder
        # Ideally kodiaq would already do this
        run_label = run_doc['name'] + '_' + run_doc['start'].strftime('%y%m%d_%H%M%S')
        
        # Get possible pax config overrides from the run doc
        conf_override = run_doc.get('pax_config', {})
        
        # Set the input (mongo collection) and output ()
        conf_override.setdefault('pax', {})
        conf_override['pax']['input_name'] = str(_id)
        conf_override['pax']['output_name'] = os.path.join(data_directory, run_label)
        
        # WARNING this will delete your data!
        conf_override.setdefault('MongoXAMS', {})
        conf_override['MongoXAMS']['delete_data'] = False
        
        print("Starting pax to process run %s" % _id)
        mypax = core.Processor(config_paths=pax_ini_config_path,
                               config_dict=conf_override)
        mypax.run()
        del mypax
        print("Pax is done!")
        
        runs_collection.find_one_and_update({'_id': _id}, {'$set': {done_field_name: True}})
        
        print("Temporary nap to prevent infinite print or something")
        time.sleep(3)

In [ ]:
runs_in_progress = list(collection.find({ 'end' : {'$exists': True}}, projection=['_id']))
if len(runs_in_progress)